

Maybe we should do it on github, but it is easy to start coding here 

[Agreed. We can do the version updates on Github later.]



**Cromosome structure:**

[ optimizer, loss, neurons_layer_1, activation_1, neurons_layer_2, activation_2,neurons_layer_3, activation_3, .... ]


For example, the chromosome:

[ sgd, sparse_categorical_crossentropy, 100, softmax, 50, softmax, 25, softmax ]

Will be equal to

```
# make a sequential model
model = keras.models.Sequential()

model.add(keras.layers.Dense(100, input_shape=(n_features,), activation="softmax"))
model.add(keras.layers.Dense(50,  activation="softmax"))
model.add(keras.layers.Dense(25,  activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

```

```
Algorithm:

  Genrate population p

```

```
Generation(p):

  for each ind in p

    fitness(ind)

    selection(p)

    crossover(p)

    mutate(p)

```




In [ ]:
#   [ optimizer, loss, neurons_layer_1, activation_1, neurons_layer_2, activation_2,neurons_layer_3, activation_3, .... ]

class Aux:

  INDEX_OPTIMIZER = 0

  INDEX_LOSS = 1

  INDEX_FIRST_HIDDEN = 2

  LOSSES_NAMES = [ "mean_absolute_error"] # "sparse_categorical_crossentropy"] # 'sparse_categorical_crossentropy']#, "categorical_crossentropy"]["binary_crossentropy", "categorical_crossentropy",

  OPTIMIZERS_NAMES = ["SGD", "Adam", "RMSprop", "Adadelta", "Adagrad", "Adamax", "Nadam", "Ftrl"]

  ACTIVATION_NAMES = ["tanh", "softmax", "sigmoid", "relu",  "softsign",  "elu" ] # "selu",, "exponential"

  SIZE_POPULATION = 20

  SELECTION_SIZE = 10

  USE_MERGE = True
  #CROSSOVER_SIZE = 5

In [ ]:
import random
import uuid

class Individual:
  def __init__(self, chromosome):
    self.id = str(uuid.uuid4())
    self.chromosome = chromosome
    self.model = self.generate_model()
    self.last_accuracy = 0
    self.parents = ("","")
    self.training_time = 1000
    self.loss = 1
    self.n_epochs = 1000
  def generate_model(self):

    print(self.chromosome)
    model = keras.models.Sequential()

    #First layer (dataset dependent)

    #model.add(keras.layers.Input(shape=(n_features,)))
    model.add(tf.keras.layers.Flatten(input_shape=input_shape))
    #model.add(keras.layers.Dense(100, activation="tanh"))
    i = Aux.INDEX_FIRST_HIDDEN

    # ['SGD', 'sparse_categorical_crossentropy', 180, 'softmax', 20, 'softmax']
    while i < len(self.chromosome):

      if i+1 == len(self.chromosome):
        # The last layer must have output shape
        model.add(keras.layers.Dense(output_shape,  activation=self.chromosome[i+1]))

      else:
        model.add(keras.layers.Dense(self.chromosome[i],  activation=self.chromosome[i+1]))

      i = i+2

    model.compile(loss=self.chromosome[Aux.INDEX_LOSS], optimizer=self.chromosome[Aux.INDEX_OPTIMIZER], metrics=["mae"]) # accuracy / mae

    keras.utils.plot_model(model, "my_mnist_model.png", show_shapes=True) #str(random.randint(0, 10000))+

    return model

In [ ]:
from random import randint
import math
def new_individual():

  loss_name = Aux.LOSSES_NAMES[randint(0, len(Aux.LOSSES_NAMES))-1]
  optimizer_name = Aux.OPTIMIZERS_NAMES[randint(0, len(Aux.OPTIMIZERS_NAMES))-1]

  chromosome = [ optimizer_name, loss_name]

  MAX_NUM_LAYERS = 5

  MAX_NUM_NEURONS = 100

  for i in range(0, randint(1, MAX_NUM_LAYERS)):
    # we want layers like: 10 - 20 - 30... not 12, or 27
    neurons = int(math.ceil(randint(10, 200) / 10.0)) * 10

    activation_name = Aux.ACTIVATION_NAMES[randint(0, len(Aux.ACTIVATION_NAMES))-1]

    chromosome.extend([neurons, activation_name])

  return Individual(chromosome)




In [ ]:
def generate_poulation():

  population = []

  for i in range(0,Aux.SIZE_POPULATION):
    population.append(new_individual())


  return population
  '''
  return [Individual([ "sgd", "sparse_categorical_crossentropy", 1000, "relu", 500, "relu", 100, "relu", 50, "softmax"]),
          Individual([ "sgd", "sparse_categorical_crossentropy", 1000, "relu", 500, "relu", 100, "relu", 50, "softmax"])] 
  '''

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time

def fitness(indv):
  
  #try:
    start_time = time.time()
    
    earlystop_callback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose=0, mode='auto', restore_best_weights=True)

    indv.model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=1000, verbose=0, callbacks=[earlystop_callback]) 

    indv.training_time = time.time() - start_time

    hist = indv.model.history.history['val_loss']
    indv.n_epochs = len(hist)

    scores = indv.model.evaluate(x_test, y_test, verbose=0)
    indv.last_accuracy = scores[1]*100
    indv.loss = scores[1] # scores[0] # 
    

  #except:
  #  print("Bad model, it will be discarded ",indv.chromosome)
  #  indv.last_accuracy = 0.0


In [ ]:
def selection(population):
  population = sorted(population, key=lambda x: x.loss, reverse=False)[:Aux.SELECTION_SIZE] # reverse = false if loss, true if accuracy
  return population

In [ ]:
# https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)#Single-point_crossover
# Uniform crossover?
# https://pynative.com/python-weighted-random-choices-with-probability/
# picking depending on the performance?
def crossover(population):
  new_population = []
  for i in range(0, Aux.SIZE_POPULATION):

    parent_a = population[randint(0, len(population)-1)] 
    parent_b = population[randint(0, len(population)-1)] 

    new_population.append(cross_parents(parent_a, parent_b))
  return new_population

In [ ]:
def cross_parents(parent_a, parent_b):

  chromosome = []
  for i in range(0, max(len(parent_a.chromosome), len(parent_b.chromosome))):
    
      
    if i >= len(parent_a.chromosome):
      chromosome.append(parent_b.chromosome[i])
    elif i >= len(parent_b.chromosome):
      chromosome.append(parent_a.chromosome[i])
    else:
      if random.random() < 0.5:
        chromosome.append(parent_a.chromosome[i])
      else:
        chromosome.append(parent_b.chromosome[i])

  mutated_chromosome = mutate_indvidual(chromosome)

  child = Individual(mutated_chromosome)
  child.parents = (parent_a.id, parent_b.id)
  if Aux.USE_MERGE:
    return weights_merges(parent_a, parent_b, child)
  else:
    return child

In [ ]:
def weights_merges(parent_a, parent_b, child):
  for idx,layer in enumerate(child.model.layers):

    if len(child.model.get_layer(index=idx).get_weights()) == 0:
      continue

    if idx >= len(parent_a.model.layers):
      target = child.model.get_layer(index=idx)
      source = parent_b.model.get_layer(index=idx)

      weight = resize_weight_matrix(source, target)

      updated_weights = target.get_weights()
      updated_weights[0] = weight
      child.model.get_layer(index=idx).set_weights(updated_weights)

    elif idx >= len(parent_b.model.layers):
      target = child.model.get_layer(index=idx)
      source = parent_a.model.get_layer(index=idx)

      weight = resize_weight_matrix(source, target)
      
      updated_weights = target.get_weights()
      updated_weights[0] = weight
      child.model.get_layer(index=idx).set_weights(updated_weights)
    else:
      target = child.model.get_layer(index=idx)
      b_layer = parent_b.model.get_layer(index=idx)
      a_layer = parent_a.model.get_layer(index=idx)

      a_weight = resize_weight_matrix(a_layer, target)
      b_weight = resize_weight_matrix(b_layer, target)

      updated_weights = target.get_weights()
      updated_weights[0] = (a_weight + b_weight)*0.5     

      child.model.get_layer(index=idx).set_weights(updated_weights)
      
  return child

In [ ]:
# Reshape source into
def resize_weight_matrix(source_model, target_model):

      first_dimension_size = max(target_model.get_weights()[0].shape[0], source_model.get_weights()[0].shape[0] )
      second_dimension_size = max(target_model.get_weights()[0].shape[1], source_model.get_weights()[0].shape[0] )

      first_dimension_size if first_dimension_size > 0 else 0

      # resize the source weights
      resized = np.pad(source_model.get_weights()[0], ((0, first_dimension_size if first_dimension_size > 0 else 0), (0, second_dimension_size if second_dimension_size > 0 else 0)), 'constant', constant_values=(0))

      return resized[0:target_model.get_weights()[0].shape[0], 0:target_model.get_weights()[0].shape[1]]

In [ ]:
def mutate_indvidual(chromosome):
  for gene_id in range(0, len(chromosome)):
    if random.random() > 0.9:
      if gene_id == Aux.INDEX_OPTIMIZER:
        copy = Aux.OPTIMIZERS_NAMES[:]
        copy.remove(chromosome[gene_id])
        chromosome[gene_id] = copy[randint(0, len(copy))-1]
      elif gene_id >= Aux.INDEX_FIRST_HIDDEN:
        if (gene_id - Aux.INDEX_FIRST_HIDDEN)%2 == 0:
          # Number Neurons
          chromosome[gene_id] = int(math.ceil(randint(10, 200) / 10.0)) * 10
        else:
          # Activation function
          copy = Aux.ACTIVATION_NAMES[:]
          copy.remove(chromosome[gene_id])
          chromosome[gene_id] = copy[randint(0, len(copy))-1]
  return chromosome
      

In [ ]:
def generation(population):
  best_result = 0.0
  for ind in population:
    fitness(ind)
    print(str(ind.id)+","+ str(ind.parents) +","+ str(ind.last_accuracy)+","+ str(ind.loss)+","+ str(ind.n_epochs)+","+str(ind.training_time))
    f.write(str(ind.id)+","+ str(ind.parents[0])+","+ str(ind.parents[1]) +","+ str(ind.last_accuracy)+","+ str(ind.loss)+","+ str(ind.n_epochs)+","+str(ind.training_time)+"\n")
    best_result = max(best_result, ind.last_accuracy)
  population = selection(population)
  population = crossover(population)
  #print("Crossover", len(population))
  #population = mutation(population)
  print("----------end of a generation--------------")
  return population, best_result

In [ ]:
def algorithm():
  p = generate_poulation()

  for i in range(10): # It shouldnt be like this, but lets leave like this at the moment
    
    new_population, best_result = generation(p)
    p = new_population



In [ ]:
## Main
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:

# Load data mnist
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

input_shape =  x_train.shape[1:] if len( x_train.shape[1:]) != 0 else 1

output_shape = 10

print("Input shape", input_shape)
print("Output shape", output_shape)




Input shape (28, 28)
Output shape 10


In [ ]:
# Load data boston_housing
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

input_shape =  x_train.shape[1:] if len( x_train.shape[1:]) != 0 else 1

output_shape = 1

print("Input shape", input_shape)
print("Output shape", output_shape)

Input shape (13,)
Output shape 1


In [ ]:
!wget https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/slr/excel/slr06.xls

--2021-01-11 00:39:48--  https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/slr/excel/slr06.xls
Resolving college.cengage.com (college.cengage.com)... 69.32.208.108
Connecting to college.cengage.com (college.cengage.com)|69.32.208.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2502 (2.4K) [application/vnd.ms-excel]
Saving to: ‘slr06.xls.2’

slr06.xls.2         100%[===================>]   2.44K  --.-KB/s    in 0s      

2021-01-11 00:39:49 (34.0 MB/s) - ‘slr06.xls.2’ saved [2502/2502]



In [ ]:
# https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/slr/frames/slr06.html

# https://github.com/RishiSD/Linear-regression-on-Swedish-Auto-Insurance-dataset/blob/master/Swedish%20Auto%20Insurance.ipynb

import pandas as pd

df = pd.read_excel('slr06.xls')



#X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

msk = np.random.rand(len(df)) < 0.8

train = df[msk]

test = df[~msk]


x_train = pd.DataFrame(train.X).to_numpy()
y_train = pd.DataFrame(train.Y).to_numpy()

x_test = pd.DataFrame(test.X).to_numpy()
y_test = pd.DataFrame(test.Y).to_numpy()

input_shape =  x_train.shape[1:] if len( x_train.shape[1:]) != 0 else 1

output_shape = 1

print("Input shape", input_shape)
print("Output shape", output_shape)

*** No CODEPAGE record, no encoding_override: will use 'ascii'
Input shape (1,)
Output shape 1


In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

--2021-01-11 00:39:49--  http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘winequality-red.csv.1’

winequality-red.csv 100%[===================>]  82.23K  --.-KB/s    in 0.1s    

2021-01-11 00:39:49 (716 KB/s) - ‘winequality-red.csv.1’ saved [84199/84199]



In [ ]:
# http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# https://machinelearningmastery.com/standard-machine-learning-datasets/
from numpy import genfromtxt
data = genfromtxt('winequality-red.csv', delimiter=';', skip_header=1)
train = data[0:1280]
test = data[0:1280]

x_train = train[:,:-1]
y_train = train[:,-1:]
x_test =  test[:,:-1]
y_test =  test[:,-1:]

input_shape =  x_train.shape[1:] if len( x_train.shape[1:]) != 0 else 1

output_shape = 1

print("Input shape", input_shape)
print("Output shape", output_shape)


Input shape (11,)
Output shape 1


In [ ]:
import os

name = "result_with_merge_wine_quality.csv"
if os.path.exists(name):
  os.remove(name)
else:
  print("The file does not exist")

f = open(name, "a")
  
f.write("id,parent_a,parent_b,accuracy,loss,n_epochs,training_time\n")
algorithm()
f.close()

The file does not exist
['Nadam', 'mean_absolute_error', 180, 'tanh', 20, 'softmax', 160, 'elu', 170, 'sigmoid']
['Adamax', 'mean_absolute_error', 70, 'relu', 150, 'relu', 130, 'softmax']
['Nadam', 'mean_absolute_error', 200, 'elu', 160, 'sigmoid', 120, 'tanh']
['Adadelta', 'mean_absolute_error', 120, 'softsign']
['Adagrad', 'mean_absolute_error', 60, 'elu']
['Adadelta', 'mean_absolute_error', 10, 'tanh', 140, 'tanh', 10, 'elu', 130, 'softsign']
['Adamax', 'mean_absolute_error', 160, 'elu', 50, 'tanh']
['Ftrl', 'mean_absolute_error', 160, 'elu', 140, 'softmax', 180, 'softsign', 10, 'elu', 150, 'sigmoid']
['Nadam', 'mean_absolute_error', 70, 'relu', 90, 'softmax', 150, 'softsign', 150, 'elu']
['Adamax', 'mean_absolute_error', 170, 'elu']
['Ftrl', 'mean_absolute_error', 40, 'softmax', 150, 'tanh', 130, 'softmax', 30, 'relu', 170, 'softsign']
['Adam', 'mean_absolute_error', 20, 'softmax', 200, 'softsign', 20, 'elu', 120, 'elu', 190, 'softsign']
['Ftrl', 'mean_absolute_error', 150, 'tanh',

In [ ]:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])

In [ ]:
    
    earlystop_callback = EarlyStopping(monitor='mae', min_delta=0.05, patience=5, verbose=1, mode='auto', restore_best_weights=True)

    model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=1000, verbose=1, callbacks=[earlystop_callback]) 

    hist = model.history.history['val_mae']

    scores = model.evaluate(x_test, y_test, verbose=0)
    #indv.last_accuracy = scores[1]*100
    #indv.loss = scores[0]
    print(model.metrics_names)
    print(scores)

Epoch 1/1000
13/13 [==============================] - 0s 8ms/step - loss: 105.1797 - mae: 8.0464 - val_loss: 82.9110 - val_mae: 7.8117
Epoch 2/1000
13/13 [==============================] - 0s 4ms/step - loss: 115.1410 - mae: 8.3193 - val_loss: 76.1959 - val_mae: 6.0899
Epoch 3/1000
13/13 [==============================] - 0s 4ms/step - loss: 76.2262 - mae: 6.6340 - val_loss: 189.3935 - val_mae: 11.1536
Epoch 4/1000
13/13 [==============================] - 0s 4ms/step - loss: 75.9414 - mae: 6.1791 - val_loss: 342.6962 - val_mae: 15.8953
Epoch 5/1000
13/13 [==============================] - 0s 4ms/step - loss: 90.2088 - mae: 6.9301 - val_loss: 54.6569 - val_mae: 5.4100
Epoch 6/1000
13/13 [==============================] - 0s 4ms/step - loss: 81.3107 - mae: 6.6711 - val_loss: 54.3951 - val_mae: 5.3032
Epoch 7/1000
13/13 [==============================] - 0s 4ms/step - loss: 84.6876 - mae: 7.0715 - val_loss: 85.1043 - val_mae: 7.9770
Epoch 8/1000
13/13 [==============================] - 0s